In [16]:

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time
from dotenv import load_dotenv
import os

#Create env variables for Spotify API
load_dotenv()
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

In [24]:
OUTPUT_PATH = 'data/final_dataset_with_library.csv'
kaggle_df = pd.read_csv('data/data.csv')
# Authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://127.0.0.1:8080",
                                               scope="user-library-read"))



In [30]:
from tqdm.auto import tqdm
def get_all_saved_tracks(sp):
    """
    Fetch all tracks from user's Spotify library with audio features
    Returns a list of track dictionaries with full metadata
    """
    saved_tracks = []
    offset = 0
    limit = 50  # Max allowed by Spotify API
    
    print("Fetching your saved tracks from Spotify...")
    
    try:
        # Get first batch to determine total
        results = sp.current_user_saved_tracks(limit=limit, offset=offset)
        total = results['total']
        
        print(f"Total tracks in your library: {total}")
        
        # Handle empty library
        if total == 0:
            print("Your library is empty!")
            return []
        
        # Use tqdm for progress bar
        with tqdm(total=total, desc="Downloading library metadata") as pbar:
            while offset < total:
                try:
                    results = sp.current_user_saved_tracks(limit=limit, offset=offset)
                    
                    # Check if we got any items
                    if not results or not results.get('items'):
                        break
                    
                    # Extract basic track info
                    for item in results['items']:
                        track = item.get('track')
                        if track and track.get('id'):
                            # Extract year from release_date
                            release_date = track.get('album', {}).get('release_date', '')
                            year = release_date.split('-')[0] if release_date else None
                            
                            saved_tracks.append({
                                'id': track['id'],
                                'name': track.get('name', 'Unknown'),
                                'artist': track['artists'][0]['name'] if track.get('artists') and len(track['artists']) > 0 else 'Unknown',
                                'album': track['album']['name'] if track.get('album') else 'Unknown',
                                'year': year,
                                'duration_ms': track.get('duration_ms'),
                                'explicit': track.get('explicit', False),
                                'popularity': track.get('popularity')
                            })
                    
                    pbar.update(len(results['items']))
                    
                    # Check if there are more tracks
                    if results['next'] is None or len(results['items']) == 0:
                        break
                    
                    offset += limit
                    time.sleep(0.1)  # Be nice to the API
                    
                except Exception as e:
                    print(f"\nError fetching tracks at offset {offset}: {e}")
                    print("Continuing with tracks fetched so far...")
                    break
        
        print(f"Successfully fetched {len(saved_tracks)} tracks from your library")
        
        # Now fetch audio features in batches
        print("\nFetching audio features for your library tracks...")
        track_ids = [track['id'] for track in saved_tracks]
        batch_size = 100  # Audio features endpoint allows up to 100 tracks
        
        all_features = []
        with tqdm(total=len(track_ids), desc="Downloading audio features") as pbar:
            for i in range(0, len(track_ids), batch_size):
                batch = track_ids[i:i+batch_size]
                try:
                    features = sp.audio_features(batch)
                    all_features.extend(features)
                    pbar.update(len(batch))
                    time.sleep(0.1)  # Be nice to the API
                except Exception as e:
                    print(f"\nError fetching audio features for batch {i//batch_size + 1}: {e}")
                    # Add None placeholders for this batch
                    all_features.extend([None] * len(batch))
                    pbar.update(len(batch))
        
        # Merge audio features with track data
        for track, features in zip(saved_tracks, all_features):
            if features:
                track['valence'] = features.get('valence')
                track['acousticness'] = features.get('acousticness')
                track['danceability'] = features.get('danceability')
                track['energy'] = features.get('energy')
                track['instrumentalness'] = features.get('instrumentalness')
                track['liveness'] = features.get('liveness')
                track['loudness'] = features.get('loudness')
                track['speechiness'] = features.get('speechiness')
                track['tempo'] = features.get('tempo')
                track['key'] = features.get('key')
                track['mode'] = features.get('mode')
                track['time_signature'] = features.get('time_signature')
            else:
                # Set to None if features couldn't be fetched
                track['valence'] = None
                track['acousticness'] = None
                track['danceability'] = None
                track['energy'] = None
                track['instrumentalness'] = None
                track['liveness'] = None
                track['loudness'] = None
                track['speechiness'] = None
                track['tempo'] = None
                track['key'] = None
                track['mode'] = None
                track['time_signature'] = None
        
        print(f"Successfully enriched {len(saved_tracks)} tracks with audio features")
        return saved_tracks
    except Exception as e:
        print(f"Error initializing library fetch: {e}")
        print("Make sure you're authenticated and have granted the correct permissions.")
        return []

def merge_datasets(kaggle_df, library_tracks):
    """
    Merge Kaggle dataset with user's library
    Add 'in_library' column
    """
    print("\nMerging datasets...")
    
    # Create set of track IDs from user's library for fast lookup
    library_ids = {track['id'] for track in library_tracks if track['id']}
    
    # Also create name+artist combinations as backup
    library_name_artist = {
        f"{track['name'].lower().strip()}||{track['artist'].lower().strip()}"
        for track in library_tracks
        if track['name'] and track['artist']
    }
    
    print(f"Library contains {len(library_ids)} unique track IDs")
    
    # Determine column names in Kaggle dataset
    possible_id_cols = ['id', 'track_id', 'spotify_id']
    possible_name_cols = ['name', 'track_name', 'song_name']
    possible_artist_cols = ['artists', 'artist', 'artist_name']
    
    id_col = next((col for col in possible_id_cols if col in kaggle_df.columns), None)
    name_col = next((col for col in possible_name_cols if col in kaggle_df.columns), None)
    artist_col = next((col for col in possible_artist_cols if col in kaggle_df.columns), None)
    
    print(f"Using columns: id='{id_col}', name='{name_col}', artist='{artist_col}'")

In [31]:
#uses spotify API to get my liked songs
#separated for efficiency
saved_tracks = get_all_saved_tracks(sp)

Fetching your saved tracks from Spotify...
Total tracks in your library: 1963


Successfully fetched 1963 tracks from your library

Fetching audio features for your library tracks...


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0DJJyOL3LfHw10WrAlOMn7,5EiLjOB57pHdNnkcrWh1m4,1a3QWkAM9UlDKOwR1wRBEn,1swz9stsbG1p34SJHJqiww,5ykbOijJEfRhuo2Td1m0Qd,0lHhXRTL75X5fDN9VsIH72,1gExMyY0hcnLisq9OJRabP,1B241LRKmK6qDDTZfUajmm,0yYQbFuqvXyZc46e2QEqI7,0q8TNCQhwnDwnzq38IVV4l,588EtZ99qVxlMWiqRnc41x,78wZn0wdzhj3imUleYWd5M,3tD1Woe9cOS3pF8eUGMIMC,0eEgMbSzOHmkOeVuNC3E0k,5cHYqe1NIq4DAzT1dSa7Zu,4TUuuk5ZIiUmQX6t97TfQU,418mJEfJMD9uspXtjFy0Kf,6scUEism6pBnY0IXYr8fw3,4VsLEqawwKRCSSxycBUg0L,5Jh1i0no3vJ9u4deXkb4aV,3iTvYySsyOjPVR3Oiw17Zq,1hoeP1oPYlhcv6iRCtB7pT,7LXN0LffItjMb9bq61htdB,3CTBfot80XAOyuaJ1jcN5z,4j2DQgFAyVuKOBFpq4vEPR,0Cn4UhpHhgf88eWaS9nn5u,0Px73m5ACfeYHYsAo9YTfP,6vmb72xFV5gq2Lg7ZRS2LB,7Gjj6NxSsVwXfOXUjvjCXv,3CQ8caFKvwGsSbgXd4ghkt,4rh7wySe2iEyEDEEzstKkt,2od6VwQ93pD6vfYss3oync,6K8VQ84MqhsoakN5MjrnVR,7m7rmf5etf4H7cyKegjGMQ,4mUR5Tv0CvJtpmnUtzeIgd,2dA8HyqpMOXYQBMbeHUQnl,1YDmjv2IFNnP3jyYaZrGOd,7EQ4XAxaCxsLbXpp3yX111,1I5AbwDbZk7UYeLUagtGXH,5Lbwx92tmkpsxKJiwnYAIZ,0AkiAfilrTU


Error fetching audio features for batch 1: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=0DJJyOL3LfHw10WrAlOMn7,5EiLjOB57pHdNnkcrWh1m4,1a3QWkAM9UlDKOwR1wRBEn,1swz9stsbG1p34SJHJqiww,5ykbOijJEfRhuo2Td1m0Qd,0lHhXRTL75X5fDN9VsIH72,1gExMyY0hcnLisq9OJRabP,1B241LRKmK6qDDTZfUajmm,0yYQbFuqvXyZc46e2QEqI7,0q8TNCQhwnDwnzq38IVV4l,588EtZ99qVxlMWiqRnc41x,78wZn0wdzhj3imUleYWd5M,3tD1Woe9cOS3pF8eUGMIMC,0eEgMbSzOHmkOeVuNC3E0k,5cHYqe1NIq4DAzT1dSa7Zu,4TUuuk5ZIiUmQX6t97TfQU,418mJEfJMD9uspXtjFy0Kf,6scUEism6pBnY0IXYr8fw3,4VsLEqawwKRCSSxycBUg0L,5Jh1i0no3vJ9u4deXkb4aV,3iTvYySsyOjPVR3Oiw17Zq,1hoeP1oPYlhcv6iRCtB7pT,7LXN0LffItjMb9bq61htdB,3CTBfot80XAOyuaJ1jcN5z,4j2DQgFAyVuKOBFpq4vEPR,0Cn4UhpHhgf88eWaS9nn5u,0Px73m5ACfeYHYsAo9YTfP,6vmb72xFV5gq2Lg7ZRS2LB,7Gjj6NxSsVwXfOXUjvjCXv,3CQ8caFKvwGsSbgXd4ghkt,4rh7wySe2iEyEDEEzstKkt,2od6VwQ93pD6vfYss3oync,6K8VQ84MqhsoakN5MjrnVR,7m7rmf5etf4H7cyKegjGMQ,4mUR5Tv0CvJtpmnUtzeIgd,2dA8HyqpMOXYQBMbeHUQnl,1YDmjv2IFNnP3jyYaZrGOd,7EQ4XAxaCxsLbXpp3yX111,1I5AbwD

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4IGT69nbtQdRXNVIZaulOf,5xHgo5JN0wfsV41HnRaos5,2bxQP85JNU11CQV4KXCnau,1FkoVC85Ds3mFoK0fVqEqP,0XYDsThQp51DSYyAX5gFc9,6srxWTZ1rRFBc6L7GUy9Rv,52H0ddjO0yDu730v5d1Df6,3UURgDYcbOzELx4jVyKbwA,3R6yNicsZrWF8ybl02imcB,3Y9cFvZjmiZCMMjTTaY5f9,3rq5VaJa4HfQIHPgGa2iH8,3Kj2EWpIBnvETsYq4cq0IH,4rlQza35DE4Prh5yonxnCs,01TnMXIy7mJJQ7E8uy937N,6J5s4zugz7DxSgAWHKtmfV,1XnLnMy8COxIVudGbLuoiH,7fB37luKPsA3Jkvr4a5iMy,71o2pLB6Xu62uVjqpwmiS9,3UEmMEcilrUnd589Nhv5ct,0FqkI0vR3Z28v6GSudbKXj,20I8RduZC2PWMWTDCZuuAN,59tskctgqUmjCWAwhzYAFm,6tNgRQ0K2NYZ0Rb9l9DzL8,6AY1M1akbsVaQN3ATVyzH7,2nMTBUdxkdCZWo7n3upy1e,4IadxL6BUymXlh8RCJJu7T,6tYGrpUyqB4yao5lAs3AR4,6nEGxVCcMhIugOHk37APU7,4Zy6m1QjUivLl8Zt7E8uxr,3hUwFkfeCUOVOMKy9iKDTw,4p9iQNEmsIGkB6eG8Val8n,7aKWgpecgLEqisWcXPElDl,4G8Eu3EDQZYhHPfJMFCv8V,2gyxAWHebV7xPYVxqoi86f,2w4cRBhWvHCsDpvyhiDkjq,4n9ifmVrDHbNdqWjlS3yLD,5T00EN4oZmDQx3hqVAC4Fx,3mmMM7hYMIMdtp2NLQITaY,2GEs9D56FRHqUAEwvlvBpw,4Ysl0ybOaEiipH0Sb3A0qy,64pGB9FHgZs

...


Error fetching audio features for batch 14: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=7f4bRMqRKRfUNPW2gbTHSt,6cDPJqKw8PUBw5SigKszrL,7DnAm9FOTWE3cUvso43HhI,359HNzfOXhCMHB1pNKhyfH,3WVhkjB7Y4xFruqoCAajBb,7Gq4tfeuWPjr6WQiB7aBXz,6zTKuNMEU3svBn65L6u88M,1SBrgGuLx1oU86u5w76Fo0,3omXshBamrREltcf24gYDC,5a6dfZPHjKE0ZdeFXMuXc7,3HWzoMvoF3TQfYg4UPszDq,3kSmaObYtsjFbusk0HBBuX,084x7MwwS1j30jFbrBJ2ib,2uoGR0vgRfviZKvmrn8n8r,0beQOSJ5aUkQky6XYlC8m6,2lchElClxgH0NVS5wQqAra,5IlQXBdOtCsrDpN7NiBgex,4PnNzWe1LJoAMD5j5RHpI0,4LUnZWnO2E2NDUOhfFJr2E,4l5PryvVh8ofU9vMDnv4qA,4cEONREpY8NQJMuGQP95Kl,0JvUjekRwmDcQq4S0Sxocf,4kV4N9D1iKVxx1KLvtTpjS,1mpgoREOBOoea0hg34AW7U,6lZe9DbvrcHZ3Ot2O9BPRc,46gwMD8DJO8B6jy2C3hEeZ,4RtexJZ1OyRkDNlqVUt8Zt,5a95ITgIWZDFNWjNU0XOfs,5WbqaQnjhI5oAN5QHvxd7x,72J8O7kyKTicwoUxuu0eYy,44uCMdE6RWnkOFYTtbmUUc,7BqHUALzNBTanL6OvsqmC1,5zxwumO0laRosoeOvbVSvc,1GO9vCm5Cra5uWGSWQ5mGK,3Jc6Bdhc4QDQwal1urpuNk,6XdJ8VDsuu0ExgecUcdfYv,7uvz3nIyTAJJJcBOcjtFyr,5LiRnQNAUI9jcH6qwbG15G,2PneyQ

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=12OyoOdomhXmlVwdDKBJaL,2pq9pe4fR25v736ZXa8IRn,4djIFfof5TpbSGRZUpsTXq,1gSsFZhkqEgQduWTNSZvys,5h7rOvESYS63Gpc175Weoc,1IoielYX6F3q06an4r5q86,1gCAWjUYxxqmJYcVhEPgB3,0iQ77tSa7ncj7zBPtVx6cU,3YQ96qkxiEwTLwgu2Izcmq,5yFSF6qQA1rcLsQRnBsZgb,0JKdYBv73Z3xqISinY7psY,4ClSEpiGyMa9YAGIXdKCQn,5MMu9zi4UrctTCXQqNJUCY,6WEl4Ke21KI4zgYjElRxFZ,5p0UXw8VPvtgGEgBltcIwU,3vAf3gMMtYSfa4DxD3hi9a,7F9vK8hNFMml4GtHsaXui6,54RfcPsxDsf5pbLigUDRpS,3dD9yyYTQ73SZvyOygyKva,0z9OOePZZiShJoL1o025bl,1FnUibXeHlApFC4Se6dtf3,1nk79W4K6owhHiXJDsr27n,2PhmGLVDm7jNvzhoV7XtcT,0uhBfy5rlLbuhDxrZfLZwP,5yX3kZyw1nM14mRiZO1Irb,0oa4VgSd2jKGU15yjtcDvr,6jqoY5HKd5CagZV5lcxdU2,4XrvRf68c9H4orZZ9KoldQ,0oJHZHmiEItS3j0L3JQzI4,3ExZOB0tZ7ADxzFdj88Ryn,6qc9V5hnDYycfwvORNrGrK,4NJFZ18M7ki6chdCJXWVZX,0AcXoZdweaXpvNedObXU9r,5RO0W6hPRdDGFNBmeyK5eY,4YusLB4dWXjKqIUGwNyKgu,1hEiVb61xXfCc9JL4cJRQP,3DntgN8cqU4feOBEFWFAoI,3D5wnLaeVlCCIPzQOOAirA,5xey3cl8y5fWst3EKgPFk8,7rUBl3RwsXg9IGPa3Lid9i,4CRvXmp3xs3


Error fetching audio features for batch 17: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=12OyoOdomhXmlVwdDKBJaL,2pq9pe4fR25v736ZXa8IRn,4djIFfof5TpbSGRZUpsTXq,1gSsFZhkqEgQduWTNSZvys,5h7rOvESYS63Gpc175Weoc,1IoielYX6F3q06an4r5q86,1gCAWjUYxxqmJYcVhEPgB3,0iQ77tSa7ncj7zBPtVx6cU,3YQ96qkxiEwTLwgu2Izcmq,5yFSF6qQA1rcLsQRnBsZgb,0JKdYBv73Z3xqISinY7psY,4ClSEpiGyMa9YAGIXdKCQn,5MMu9zi4UrctTCXQqNJUCY,6WEl4Ke21KI4zgYjElRxFZ,5p0UXw8VPvtgGEgBltcIwU,3vAf3gMMtYSfa4DxD3hi9a,7F9vK8hNFMml4GtHsaXui6,54RfcPsxDsf5pbLigUDRpS,3dD9yyYTQ73SZvyOygyKva,0z9OOePZZiShJoL1o025bl,1FnUibXeHlApFC4Se6dtf3,1nk79W4K6owhHiXJDsr27n,2PhmGLVDm7jNvzhoV7XtcT,0uhBfy5rlLbuhDxrZfLZwP,5yX3kZyw1nM14mRiZO1Irb,0oa4VgSd2jKGU15yjtcDvr,6jqoY5HKd5CagZV5lcxdU2,4XrvRf68c9H4orZZ9KoldQ,0oJHZHmiEItS3j0L3JQzI4,3ExZOB0tZ7ADxzFdj88Ryn,6qc9V5hnDYycfwvORNrGrK,4NJFZ18M7ki6chdCJXWVZX,0AcXoZdweaXpvNedObXU9r,5RO0W6hPRdDGFNBmeyK5eY,4YusLB4dWXjKqIUGwNyKgu,1hEiVb61xXfCc9JL4cJRQP,3DntgN8cqU4feOBEFWFAoI,3D5wnLaeVlCCIPzQOOAirA,5xey3c

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=14oTuzcNwzAsBqzv5NRv6U,7AhlMDKQelsNkWvFeZvmtC,1wdAHxgIaXECOYOHgseniK,3mVQUlrhWN5Z1i1HUhU8ZH,2eAAEa8pxKF7My0EO4rFgR,6s2KvPc76EAkOuCg9fPixm,3P2CQ3VbkRJilviCuOGZl4,61WR5FfGNLJxpLsAc61qWQ,3eR23VReFzcdmS7TYCrhCe,7kHaGMriSF6GLLWMtub5Q0,6jA8HL9i4QGzsj6fjoxp8Y,7Aik6WAROo330QJVCJZUeh,5EqjEnNVbolSj52gNf7fFT,4rHmKlFRiFzabiVO6e9w2e,2Ytv0dhe5vPTJsC9pO7VO0,1vvNmPOiUuyCbgWmtc6yfm,6t2ubAB4iSYOuIpRAOGd4t,6s3GEN8wK0OMzzzZbXj0fu,2syQ0f4SAQdzvlAewPl1DJ,7GZKme55JpmLq3alBDQm11,6RotBIo70VVzEcOhvgJGPC,27PmvZoffODNFW2p7ehZTQ,5cF0dROlMOK5uNZtivgu50,5n0CTysih20NYdT2S0Wpe8,7nD9nN3jord9wWcfW3Gkcm,5masKPHeAOVNgxdLebIcK7,7q0cXzFMcNVmCvBU3mZveQ,77U6kcldLRFl9vCfaw11Sr,2Ae5awwKvQpTBKQHr1TYCg,1Rl40PsX4mh9xVOeGHq86B,1KGi9sZVMeszgZOWivFpxs,7gm2omqXDTaT4sS17hSu8T,5Iq6gR32JDFmF9RYmMpAGz,6uBhi9gBXWjanegOb2Phh0,2qV4oFPfGjIH4bg9L8mjkJ,5o3GnrcFtvkdf3zFznuSbA,2NlTOhsAamXOaZciOXbITb,3caOP7cCdKv5VFGWptOcUw,4DZghpw50ZnO3ckfDuNkft,27zK2lkAiqFw7y113I94H5,0pSBuHjILhN


Error fetching audio features for batch 18: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=14oTuzcNwzAsBqzv5NRv6U,7AhlMDKQelsNkWvFeZvmtC,1wdAHxgIaXECOYOHgseniK,3mVQUlrhWN5Z1i1HUhU8ZH,2eAAEa8pxKF7My0EO4rFgR,6s2KvPc76EAkOuCg9fPixm,3P2CQ3VbkRJilviCuOGZl4,61WR5FfGNLJxpLsAc61qWQ,3eR23VReFzcdmS7TYCrhCe,7kHaGMriSF6GLLWMtub5Q0,6jA8HL9i4QGzsj6fjoxp8Y,7Aik6WAROo330QJVCJZUeh,5EqjEnNVbolSj52gNf7fFT,4rHmKlFRiFzabiVO6e9w2e,2Ytv0dhe5vPTJsC9pO7VO0,1vvNmPOiUuyCbgWmtc6yfm,6t2ubAB4iSYOuIpRAOGd4t,6s3GEN8wK0OMzzzZbXj0fu,2syQ0f4SAQdzvlAewPl1DJ,7GZKme55JpmLq3alBDQm11,6RotBIo70VVzEcOhvgJGPC,27PmvZoffODNFW2p7ehZTQ,5cF0dROlMOK5uNZtivgu50,5n0CTysih20NYdT2S0Wpe8,7nD9nN3jord9wWcfW3Gkcm,5masKPHeAOVNgxdLebIcK7,7q0cXzFMcNVmCvBU3mZveQ,77U6kcldLRFl9vCfaw11Sr,2Ae5awwKvQpTBKQHr1TYCg,1Rl40PsX4mh9xVOeGHq86B,1KGi9sZVMeszgZOWivFpxs,7gm2omqXDTaT4sS17hSu8T,5Iq6gR32JDFmF9RYmMpAGz,6uBhi9gBXWjanegOb2Phh0,2qV4oFPfGjIH4bg9L8mjkJ,5o3GnrcFtvkdf3zFznuSbA,2NlTOhsAamXOaZciOXbITb,3caOP7cCdKv5VFGWptOcUw,4DZghp

In [29]:
print(saved_tracks)
print(kaggle_df.columns)

[{'id': '0DJJyOL3LfHw10WrAlOMn7', 'name': 'Three Words Away', 'artist': 'Orla Gartland', 'album': 'Everybody Needs a Hero'}, {'id': '5EiLjOB57pHdNnkcrWh1m4', 'name': 'Sunrise', 'artist': 'Childish Gambino', 'album': 'Camp'}, {'id': '1a3QWkAM9UlDKOwR1wRBEn', 'name': 'BRIGHTSIDE', 'artist': 'The Lumineers', 'album': 'BRIGHTSIDE'}, {'id': '1swz9stsbG1p34SJHJqiww', 'name': 'Murder on the Dancefloor - triple j Like A Version', 'artist': 'Royel Otis', 'album': 'Murder on the Dancefloor (triple j Like A Version)'}, {'id': '5ykbOijJEfRhuo2Td1m0Qd', 'name': '7', 'artist': 'Catfish and the Bottlemen', 'album': 'The Ride'}, {'id': '0lHhXRTL75X5fDN9VsIH72', 'name': 'Wet', 'artist': 'Annie DiRusso', 'album': 'Wet'}, {'id': '1gExMyY0hcnLisq9OJRabP', 'name': 'Magic Mountain', 'artist': 'Hollow Ship', 'album': 'Future Remains'}, {'id': '1B241LRKmK6qDDTZfUajmm', 'name': 'Cocoon', 'artist': 'Catfish and the Bottlemen', 'album': 'The Balcony'}, {'id': '0yYQbFuqvXyZc46e2QEqI7', 'name': 'Afterlife', 'artis

In [27]:
#merges datasets together and and saves to csv

df_final = merge_datasets(kaggle_df, saved_tracks)

df_final.to_csv(OUTPUT_PATH, index=False)


Merging datasets...
Library contains 1963 unique track IDs
Using columns: id='id', name='name', artist='artists'


AttributeError: 'NoneType' object has no attribute 'to_csv'